#GrapeVine Disease Analysis


## 1. Install and become one with the data

This project is intended to predict the disease in a grapevine, given the image. The [Grapevine Disease Images](https://www.kaggle.com/datasets/piyushmishra1999/plantvillage-grape) dataset from kaggle, will be used

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/grapevine_disease_images.zip')
zip_ref.extractall()

In [ ]:
#Lets see the amount of images and directories
import os
for dirpath, dirnames, filenames in os.walk('/content/Grape'):
  print(f'There are {len(dirnames)} directories and {len(filenames)} images in {dirpath}')

There are 4 directories and 0 images in /content/Grape
There are 0 directories and 423 images in /content/Grape/Grape___healthy
There are 0 directories and 1076 images in /content/Grape/Grape___Leaf_blight_(Isariopsis_Leaf_Spot)
There are 0 directories and 1383 images in /content/Grape/Grape___Esca_(Black_Measles)
There are 0 directories and 1180 images in /content/Grape/Grape___Black_rot


## 2. Preprocess the data and create train and test batches

In [ ]:
import tensorflow as tf

#Create data_dir
data_dir = '/content/Grape'

#Create train and test data
train_data = tf.keras.preprocessing.image_dataset_from_directory(data_dir,
                                                                 image_size=(224, 224),
                                                                 validation_split=0.2,
                                                                 subset='training',
                                                                 seed=120)
test_data = tf.keras.preprocessing.image_dataset_from_directory(data_dir,
                                                                image_size=(224, 224),
                                                                validation_split=0.2,
                                                                subset='validation',
                                                                seed=120)

Found 4062 files belonging to 4 classes.
Using 3250 files for training.
Found 4062 files belonging to 4 classes.
Using 812 files for validation.


## 3. Creating Multiple model experiments

- `model0` :- Creating a simple Convolutional neural network model
- `model1` :- Creating a Convolutional neural network model with data augmentation
- `model2` :- Create a EfficientNetV2B0 feature extraction transfer learning model


In [ ]:
#Import all necessary libraries
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd 
import numpy as np

###Model0

In [ ]:
#Create the CNN
model0 = tf.keras.Sequential([
    Conv2D(10, 3, input_shape=(224, 224, 3)),
    MaxPool2D(),
    Conv2D(10, 3, input_shape=(224, 224, 3)),
    MaxPool2D(),
    Conv2D(10, 3, input_shape=(224, 224, 3)),
    MaxPool2D(),
    Flatten(),
    Dense(4, activation='softmax')
])

#Compile our CNN
model0.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               optimizer=Adam(),
               metrics=['accuracy'])

In [ ]:
history0 = model0.fit(train_data,
                      epochs=5,
                      steps_per_epoch=len(train_data),
                      validation_data=test_data,
                      validation_steps=len(test_data),
                      callbacks=tf.keras.callbacks.TensorBoard('Experiments/Model0'))

Epoch 1/5
102/102 [==============================] - 15s 64ms/step - loss: 27.3528 - accuracy: 0.6729 - val_loss: 3.2518 - val_accuracy: 0.7906
Epoch 2/5
102/102 [==============================] - 6s 60ms/step - loss: 2.0280 - accuracy: 0.8532 - val_loss: 1.6768 - val_accuracy: 0.8584
Epoch 3/5
102/102 [==============================] - 5s 48ms/step - loss: 0.9674 - accuracy: 0.9025 - val_loss: 1.5914 - val_accuracy: 0.8522
Epoch 4/5
102/102 [==============================] - 6s 61ms/step - loss: 0.3501 - accuracy: 0.9498 - val_loss: 1.5211 - val_accuracy: 0.8633
Epoch 5/5
102/102 [==============================] - 5s 47ms/step - loss: 0.2184 - accuracy: 0.9643 - val_loss: 1.4111 - val_accuracy: 0.8682


In [ ]:
#Evaluating model0
model0.evaluate(test_data)

26/26 [==============================] - 1s 29ms/step - loss: 1.4111 - accuracy: 0.8682


[1.4111162424087524, 0.8682265877723694]

In [ ]:
model0.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 10)      280       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 10)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 10)      910       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 10)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 10)        910       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 26, 26, 10)       0

Model0 score :- About 88%

###Model1

In [ ]:
#Create CNN with data augmentation layers
model1 = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    Conv2D(10, 3, input_shape=(224, 224, 3)),
    MaxPool2D(),
    Conv2D(10, 3, input_shape=(224, 224, 3)),
    MaxPool2D(),
    Conv2D(10, 3, input_shape=(224, 224, 3)),
    MaxPool2D(),
    Flatten(),
    Dense(4, activation='softmax')
])

model1.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               optimizer=Adam(),
               metrics=['accuracy'])

In [ ]:
history_1 = model1.fit(train_data,
                       epochs=5,
                       steps_per_epoch=len(train_data),
                       validation_data=test_data,
                       validation_steps=len(test_data),
                       callbacks=tf.keras.callbacks.TensorBoard('Experiments/Model1'))

Epoch 1/5
102/102 [==============================] - 11s 102ms/step - loss: 43.8937 - accuracy: 0.5778 - val_loss: 9.3449 - val_accuracy: 0.6478
Epoch 2/5
102/102 [==============================] - 10s 93ms/step - loss: 4.7945 - accuracy: 0.7178 - val_loss: 5.5949 - val_accuracy: 0.7155
Epoch 3/5
102/102 [==============================] - 10s 91ms/step - loss: 4.2008 - accuracy: 0.7305 - val_loss: 4.1842 - val_accuracy: 0.7500
Epoch 4/5
102/102 [==============================] - 10s 100ms/step - loss: 3.3178 - accuracy: 0.7640 - val_loss: 5.6724 - val_accuracy: 0.7106
Epoch 5/5
102/102 [==============================] - 9s 84ms/step - loss: 2.3837 - accuracy: 0.7889 - val_loss: 4.0851 - val_accuracy: 0.7229


In [ ]:
model1.evaluate(test_data)

26/26 [==============================] - 1s 28ms/step - loss: 4.0851 - accuracy: 0.7229


[4.085053443908691, 0.7229064106941223]

In [ ]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 224, 224, 3)       0         
                                                                 
 random_rotation (RandomRota  (None, 224, 224, 3)      0         
 tion)                                                           
                                                                 
 conv2d_3 (Conv2D)           (None, 222, 222, 10)      280       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 10)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 10)      910       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 10)      

Model1 Score :- 75%

###Model2

In [ ]:
#1. Create the base model
base_model = tf.keras.applications.EfficientNetV2B0(include_top=False)

#2. Freeze the layers
base_model.trainable = False

#3. Create inputs
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs)

#4. Add a global pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)

#5. Create output layer
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

#6. Create the model
model2 = tf.keras.Model(inputs, outputs)

24274472/24274472 [==============================] - 2s 0us/step


In [ ]:
#7. Compile
model2.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
               optimizer=Adam(),
               metrics=['accuracy'])

#8. Fit data
history_2 = model2.fit(train_data,
                       epochs=5,
                       steps_per_epoch=len(train_data),
                       validation_data=test_data,
                       validation_steps=len(test_data),
                       callbacks=tf.keras.callbacks.TensorBoard('Experiments/Model2'))

Epoch 1/5
102/102 [==============================] - 17s 100ms/step - loss: 0.5742 - accuracy: 0.8049 - val_loss: 0.2647 - val_accuracy: 0.9470
Epoch 2/5
102/102 [==============================] - 8s 80ms/step - loss: 0.2220 - accuracy: 0.9489 - val_loss: 0.1643 - val_accuracy: 0.9594
Epoch 3/5
102/102 [==============================] - 8s 80ms/step - loss: 0.1562 - accuracy: 0.9612 - val_loss: 0.1230 - val_accuracy: 0.9667
Epoch 4/5
102/102 [==============================] - 8s 75ms/step - loss: 0.1227 - accuracy: 0.9726 - val_loss: 0.0979 - val_accuracy: 0.9741
Epoch 5/5
102/102 [==============================] - 8s 78ms/step - loss: 0.0999 - accuracy: 0.9772 - val_loss: 0.0839 - val_accuracy: 0.9791


In [ ]:
model2.evaluate(test_data)

26/26 [==============================] - 2s 61ms/step - loss: 0.0839 - accuracy: 0.9791


[0.08393459767103195, 0.9790640473365784]

In [ ]:
model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, None, None, 1280)  5919312  
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_2 (Dense)             (None, 4)                 5124      
                                                                 
Total params: 5,924,436
Trainable params: 5,124
Non-trainable params: 5,919,312
_________________________________________________________________


Model2 score :- 98%

Woah!!! Model 2 just beat both our previous models drastically!!

## Evaluation

In [ ]:
!tensorboard dev upload --logdir '/content/Experiments'


***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

/content/Experiments

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=373649185512-8v619h5kft38l4456nm2dj4ubeqsrvh6.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email&state=xsN7UgdWGtdTn46nbaR7Vclxxew3sN&prompt=consent&access_type=

## Conclusion

In conclusion here are the results for all the experiments
- `model0` validation accuracy :- 88%
- `model1` validation accuracy :- 75%
- `model2` validation accuracy :- 98%

`model2` achieved the highest accuracy of 98%, with a Sparse categorical croo entropy loss score of 0.08